In [ ]:
import torch, baukit
from transformers import AutoModelForCausalLM, AutoTokenizer
#MODEL_NAME = "gpt2-xl"  # gpt2-xl or EleutherAI/gpt-j-6B
MODEL_NAME = "gpt2-xl"
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=False).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME)
)
baukit.set_requires_grad(False, model)

In [ ]:
import baukit
baukit.set_requires_grad(False, model)

In [ ]:
def get_hidden_states_keyspace(model, tok, prefix):
    import re
    from baukit import TraceDict
    inp = {k: torch.tensor(v)[None].cuda() for k, v in tok(prefix).items()}
    print(inp)
    layer_names = [n for n,_ in model.named_modules() if 'c_fc' in n]
    with TraceDict(model, layer_names) as tr:
        logits = model(**inp)['logits']
    return torch.stack([tr[layername].output[0] for layername in layer_names])

prompt = 'Hello, my name is also'
hs = get_hidden_states_keyspace(model, tok, prompt)
hs.shape

In [ ]:
model.transformer.h[0].mlp.c_fc.weight.shape

In [ ]:
model